## Языковое моделирование

### Импорты

In [1]:
import numpy as np
import tensorflow as tf

C:\Users\User\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Настройки

In [2]:
PATH = './evgenyi_onegin.txt'

SEQ_LENGTH = 100

EPOCHS = 10
BATCH_SIZE = 64
BUFFER_SIZE = 20000

RNN_UNITS = 1024
EMBEDDING_DIM = 256

### Данные

In [3]:
with open(PATH, 'r', encoding='utf-8') as file:
    text = file.read()

text = '\n'.join(line.strip() for line in text.split('\n'))

print(len(text))
print(text[:500])

143928
Александр Сергеевич Пушкин

Евгений Онегин
Роман в стихах

Не мысля гордый свет забавить,
Вниманье дружбы возлюбя,
Хотел бы я тебе представить
Залог достойнее тебя,
Достойнее души прекрасной,
Святой исполненной мечты,
Поэзии живой и ясной,
Высоких дум и простоты;
Но так и быть - рукой пристрастной
Прими собранье пестрых глав,
Полусмешных, полупечальных,
Простонародных, идеальных,
Небрежный плод моих забав,
Бессонниц, легких вдохновений,
Незрелых и увядших лет,
Ума холодных наблюдений
И сердца го


In [4]:
vocab = sorted(set(text))
char2idx = {char: i for i, char in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[char] for char in text])
print(len(text_as_int), len(text))

143928 143928


In [5]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(SEQ_LENGTH+1, drop_remainder=True)
dataset = sequences.map(split_input_target)
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

### Модель

In [6]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
                                 
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [7]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [8]:
vocab_size = len(vocab)
model = build_model(vocab_size, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.compile(optimizer='adam', loss=loss)

In [9]:
history = model.fit(dataset, epochs=EPOCHS)

Epoch 1/10
22/22 [==============================] - 149s 7s/step - loss: 4.3099
Epoch 2/10
22/22 [==============================] - 151s 7s/step - loss: 3.4238
Epoch 3/10
22/22 [==============================] - 151s 7s/step - loss: 3.1087
Epoch 4/10
22/22 [==============================] - 153s 7s/step - loss: 2.7958
Epoch 5/10
22/22 [==============================] - 153s 7s/step - loss: 2.5761
Epoch 6/10
22/22 [==============================] - 152s 7s/step - loss: 2.4743
Epoch 7/10
22/22 [==============================] - 173s 8s/step - loss: 2.3957
Epoch 8/10
22/22 [==============================] - 171s 8s/step - loss: 2.3229
Epoch 9/10
22/22 [==============================] - 167s 8s/step - loss: 2.2518
Epoch 10/10
22/22 [==============================] - 159s 7s/step - loss: 2.1825


In [24]:
def generate_text(model, start_string):
  
    num_generate = 512
    input_eval = [char2idx[char] for char in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []
  
    temperature = 0.7

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
       
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [25]:
gen_text = generate_text(model, start_string="Случайный текст ")
print(len(gen_text), gen_text, sep='\n')

528
Случайный текст вебой гА воснажу
Уной на
Са,
Внетой ропона; это сней тай со дой,
Есто.
Не
Вон нь вист вы, та
Вой пей нащуло
Ва;
Спослда праль,
И в и дон да нет ной пы,
Чата
Ота, да по, нота"bL
Поюmf. талымох оторего ворасле орей
2
Пе ска пиро, у.
Са
Па омору
На,
Вой праса во бро.
Кране.
Нохли по, ц метойвытот простой о мотлотой сухот зажатахы
Мащу моми нины, ой
Грой ду зарода;
Вы и иы стдойшамоцала бодетоли ли вя ци, щемы тали,
Нока кохи во,
Иело о са вотуНо
Истойсту
Пло бомомваку
Прямам выхохьяне но, и Лины
Очаногарь?
Те,
